# Pagerank based Information retrieval system
In this homework we'll have to deal with the PageRank algorithm.

Import the needed Python packages.

In [ ]:
!pip install --upgrade --no-cache-dir gdown
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from itertools import combinations
!pip install scikit-network
from sknetwork.ranking import PageRank

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


The data we need to process comes from the book *Le Morte D'Arthur* by Thomas Malory.
The dataset we need to build should be an unweighted and undirected graph, where nodes represent characters from the book and an edge connects two characters in the graph if their names appeared at least one time in the same chapter.

Using this dataset, we then run various PageRank algorithms.

### 1.1
Download the data from the Drive link.

In [ ]:
!gdown 1zHgvidy9FvhZvE68S0mXWkoF-hHMpiUL
!gdown 1VjpTkFcbfaLIi4TXVafokW9e_bvGnfut

Downloading...
From: https://drive.google.com/uc?id=1zHgvidy9FvhZvE68S0mXWkoF-hHMpiUL
To: /content/The Project Gutenberg eBook of Le Morte D’Arthur, Volume I (of II), by Thomas Malory.html
100% 964k/964k [00:00<00:00, 562MB/s]
Downloading...
From: https://drive.google.com/uc?id=1VjpTkFcbfaLIi4TXVafokW9e_bvGnfut
To: /content/The Project Gutenberg eBook of Le Morte D’Arthur, Volume II (of II), by Thomas Malory.html
100% 1.10M/1.10M [00:00<00:00, 579MB/s]


### 1.2
Parse the HTML. **Part** of code already provided: follow the comments to complete the code.


In [ ]:
with open('The Project Gutenberg eBook of Le Morte D’Arthur, Volume I (of II), by Thomas Malory.html') as fp:
    vol1 = BeautifulSoup(fp, 'html.parser')
with open('The Project Gutenberg eBook of Le Morte D’Arthur, Volume II (of II), by Thomas Malory.html') as fp:
    vol2 = BeautifulSoup(fp, 'html.parser')

def clean_text(txt):
    words_to_put_space_before = [".",",",";",":","’","'"]
    words_to_lowercase = ["First","How","Some","Yet","Of","A","The","What","Fifth"]
    
    app = txt.replace("\n"," ")
    for word in words_to_put_space_before:
        app = app.replace(word," "+word)
    for word in words_to_lowercase:
        app = app.replace(word+" ",word.lower()+" ")
    return app.strip()

def parse_html(soup):
    titles = []
    texts = []
    for chapter in soup.find_all("h3"):
        chapter_title = chapter.text
        if "CHAPTER" in chapter_title:
            chapter_title = clean_text("".join(chapter_title.split(".")[1:]))
            titles.append(chapter_title)
            
            chapter_text = [p.text for p in chapter.findNextSiblings("p")]
            chapter_text = clean_text(" ".join(chapter_text))
            texts.append(chapter_text)
    return titles, texts

In [ ]:
titles1, texts1 = parse_html(vol1)

titles2, texts2 = parse_html(vol2)


#Transform the list into a pandas DataFrame.

df = pd.DataFrame({'title': titles1, 'text': texts1, 'vol': 1})

temp = pd.DataFrame({'title': titles2, 'text': texts2, 'vol': 2})

df = pd.concat([df, temp], ignore_index=True)

del temp

Print the first 8 rows of the DataFrame.

[comment]: <> (#SHOW_CELL#)

In [ ]:
display(df.head(8))

,title,text,vol
0,"first , how Uther Pendragon sent for the duke ...","It befell in the days of Uther Pendragon , whe...",1
1,how Uther Pendragon made war on the duke of Co...,"Then Ulfius was glad , and rode on more than a...",1
2,of the birth of King Arthur and of his nurture,Then Queen Igraine waxed daily greater and gre...,1
3,of the death of King Uther Pendragon,Then within two years King Uther fell sick of ...,1
4,"how Arthur was chosen king , and of wonders an...",Then stood the realm in great jeopardy long wh...,1
5,how King Arthur pulled out the sword divers times,"Now assay , said Sir Ector unto Sir Kay . And ...",1
6,"how King Arthur was crowned , and how he made ...",And at the feast of Pentecost all manner of me...,1
7,"how King Arthur held in Wales , at a Pentecost...","Then the king removed into Wales , and let cry...",1


<div style="page-break-after: always; visibility: hidden">
\pagebreak
</div>

### 1.3
Extract character's names from the **titles** only.

In [ ]:
all_characters = set()
def extract_character_names_from_string(string_to_parse):
    special_tokens = ["of","the","le","a","de"]

    remember = ""
    last_is_special_token = False

    tokens = string_to_parse.split(" ")
    characters_found = set()
    for i,word in enumerate(tokens):
        if word[0].isupper() or (remember!="" and word in special_tokens):
            #word = word.replace("'s","").replace("’s","")
            last_is_special_token = False
            if remember!="":
                if word in special_tokens:
                    last_is_special_token = True
                remember = remember+" "+word
            else: remember = word
        else:
            if remember!="":
                if last_is_special_token:
                    for tok in special_tokens:
                        remember = remember.replace(" "+tok,"")
                characters_found.add(remember)
            remember = ""
            last_is_special_token = False
    return characters_found

#all_characters = set([x for x in all_characters if x[-2:]!="'s"])

In [ ]:
#Extract all characters' names

all_characters = set().union(*list(df.title.apply(extract_character_names_from_string)))
print(all_characters)

{'Sir Galahad', 'King Arthur', 'Sir Beaumains', 'Ettard', 'Great Royalty', 'Griflet', 'Saracens', 'Palomides', 'Beale Pilgrim', 'Sir Pelleas', 'King Solomon', 'King', 'Sir Frol', 'King Bors', 'Knight of the Red Launds', 'Sir Uriens', 'Sir Brian', 'Anglides', 'Sir Pervivale', 'Joyous Isle', 'Sir Tristram de Liones', 'Sir Agravaine', 'King Lot', 'King of the Land of Cameliard', 'Beale Isoud', 'Pelles', 'King Pellam', 'King Howel of Brittany', 'Rome', 'Gawaine', 'Sir Kay', 'Cornwall', 'Lady of the Lake', 'Sir Accolon of Gaul', 'Knight of the Black Launds', 'Sangreal', 'Lionel', 'Tintagil', 'Sir Urre', 'Leodegrance', 'Carlion', 'La Cote Male Taile', 'King Mark', 'Queen of Orkney', 'Maid of Astolat', 'Beaumains', 'Sir Palomides', 'Sir Gawaine', 'Sir Marhaus', 'Ulfius', 'Queen Igraine', 'Sir Dagonet', 'Surluse', 'Sir Pedivere', 'Sir Lavaine', 'Sir Bedivere', 'York', 'Maiden of the Lake', 'France', 'King Leodegrance', 'Maledisant', 'Andred', 'Epinogris', 'Forest Perilous', 'Solomon', 'Abbot',

Print the names of all the kings (i.e. characters with `King` in their name).

[comment]: <> (#SHOW_CELL#)

In [ ]:
for name in all_characters:
  if "King" in name:
    print(name)

King Arthur
King Solomon
King
King Bors
King Lot
King of the Land of Cameliard
King Pellam
King Howel of Brittany
King Mark
King Leodegrance
King Pelleas
King Evelake
Maimed King
King Pellinore
King Ban
King Brandegore
King of England
King Pelles
King Lot of Orkney
King Bagdemagus
King Mordrains
King Anguish of Ireland
King Uriens
King Rience
King Mark of Cornwall


<div style="page-break-after: always; visibility: hidden">
\pagebreak
</div>

### 1.4
Some names refer to the same characters (e.g. `'Arthur' = 'King Arthur'`). A function is provided to extract the disambiguation dictionary: each key represents a name and the value represents the true character name (e.g. `{'Arthur': 'King-Arthur', 'King': 'King-Arthur', 'Bedivere':'Sir Bedivere'}`). Disambiguation sets, i.e. a list with sets representing the multiple names of a single character, are also provided.

> There may be some mistakes, but it does not matter (e.g. `'Cornwall' = 'King of Cornwall'`)

In [ ]:
disambiguate_to = {}
for x in all_characters:
    for y in all_characters:
        if x in y and x!=y:
            if x in disambiguate_to:
                previous_y = disambiguate_to[x]
                if len(y)>len(previous_y): disambiguate_to[x] = y
            else:
                disambiguate_to[x] = y
disambiguate_to.update({"King": "King Arthur",
                        "King of England": "King Arthur",
                        "Queen": "Queen Guenever",
                        "Sir Lancelot": "Sir Launcelot"})

disambiguate_sets = []
for x,y in disambiguate_to.items():
    inserted = False
    for z in disambiguate_sets:
        if x in z or y in z:
            z.add(x); z.add(y)
            inserted = True
    if not inserted:
        disambiguate_sets.append(set([x,y]))

while True:
    to_remove,to_add = [],[]
    for i1,s1 in enumerate(disambiguate_sets[:-1]):
        for s2 in disambiguate_sets[i1+1:]:
            if len(s1.intersection(s2))>0:
                to_remove.append(s1)
                to_remove.append(s2)
                to_add.append(s1.union(s2))
    if len(to_add)>0:
        for rm in to_remove:
            disambiguate_sets.remove(rm)
        for ad in to_add:
            disambiguate_sets.append(ad)
    else: break

### 1.5
Prepare the dataset for the PageRank algorithm.

> It will be a Pandas DataFrame with two fields: `character_1`, `character_2`.

> Each row contains two characters' names if they appear together in at least one chapter **text**.

> The relevant characters are only those extracted in Part 1.1.3.

> Keep in mind that some characters have alternative names, but they refer to the same character.

> The dataset does not contain repetitions.

In [ ]:
names = disambiguate_sets.copy()  
for name in tqdm(all_characters): # we add to disambiguate_sets the names without nicknames
    ended = False
    for i, nick_names in enumerate(disambiguate_sets):
        if name in nick_names: 
            break
        if i == len(disambiguate_sets)-1 and name not in nick_names:
            ended = True
    if ended:
        names.append({name})
# create a dictionary as keys the largest representative names for all possible ambiguities
name_dict = dict(zip( list(map(lambda x: max(x, key=len), names)) , names))
couples = []
# for each combination of characters check if they appear in the same text at least one time
for char1, char2 in tqdm(list(combinations(list(name_dict.keys()), 2))):
  combs = [(n1, n2) for n1 in name_dict[char1] for n2 in name_dict[char2]] # we also check for the possible nicknames

  control = False
  for n1, n2 in combs:
    for text in df.text:
      if n1 in text and n2 in text:
        control = True
        couples.append((char1, char2))
        break
    if control: break
# data frame with all the couples of characters
couples_df = pd.DataFrame({'character_1': [x[0] for x in couples], 'character_2': [x[1] for x in couples]})

  0%|          | 0/225 [00:00<?, ?it/s]

  0%|          | 0/14365 [00:00<?, ?it/s]

Print the rows of the dataset where `Sir Lamorak` appears.

[comment]: <> (#SHOW_CELL#)

In [ ]:
display(couples_df[(couples_df.character_1.str.contains('Sir Lamorak')) | (couples_df.character_2.str.contains('Sir Lamorak')) ])

,character_1,character_2
21,Lady Ettard,Sir Lamorak de Galis
89,Sir Palomides,Sir Lamorak de Galis
186,King Lot of Orkney,Sir Lamorak de Galis
251,La Beale Isoud,Sir Lamorak de Galis
393,Sir Gawaine,Sir Lamorak de Galis
...,...,...
2543,Sir Lamorak de Galis,Castle of Maidens
2544,Sir Lamorak de Galis,Sir Berluse
2545,Sir Lamorak de Galis,Red Knight
2546,Sir Lamorak de Galis,Sir Blamore


<div style="page-break-after: always; visibility: hidden"> 
\pagebreak
</div>

### 1.6
Print the sorted list of all character names (without duplicates) in ascending alphabetical order.

Print also the length of this list.

In [ ]:
sorted_characters = sorted(list(name_dict.keys()))

print("The sorted list with all character names (without duplicates):\n", sorted_characters)

print()
print("The length of the list:", len(sorted_characters))

The sorted list with all character names (without duplicates):
 ['Abbot', 'Alice', 'Alisander le Orphelin', 'Almaine', 'Almesbury', 'Andred', 'Anglides', 'Archbishop of Canterbury', 'Avoutres', 'Balan', 'Balin', 'Beale Pilgrim', 'Benwick', 'Boudwin', 'Bragwaine', 'Camelot', 'Carbonek', 'Carlion', 'Castle Lonazep', 'Castle of Maidens', 'Castle of Pendragon', 'Chapel Perilous', 'Christmas', 'Constantine', 'Corsabrin', 'Court', 'Dame Brisen', 'Dame Elaine', 'Damosel of the Lake', 'David', 'Dover', 'Excalibur', 'Fair Maid of Astolat', 'Feast of Pentecost', 'Forest Perilous', 'France', 'Garlon', 'God', 'Gouvernail', 'Great Royalty', 'Griflet', 'Helin le Blank', 'Holy Sangreal', 'Humber', 'Island', 'Joseph', 'Joyous Gard', 'Joyous Isle', 'Kehydius', 'King Anguish of Ireland', 'King Bagdemagus', 'King Ban', 'King Bors', 'King Brandegore', 'King Evelake', 'King Howel of Brittany', 'King Leodegrance', 'King Lot of Orkney', 'King Mark of Cornwall', 'King Mordrains', 'King Pellam', 'King Pelleas'

<div style="page-break-after: always; visibility: hidden">
\pagebreak
</div>

### 1.7
Create the adjacency matrix for the graph, assigning to each character a node identifier equal to the index that the character name has in ascending alphabetical order.

In [ ]:
encoded_df = pd.DataFrame()
# create data frame in wich we map the name of the character for his position in the sorted list
encoded_df['character_1'] = couples_df.character_1.apply(sorted_characters.index)
encoded_df['character_2'] = couples_df.character_2.apply(sorted_characters.index)
# create an encoded list for all the characters that do not appear in couples
encoded_list = list(map(sorted_characters.index, sorted_characters))

encoded_df = pd.concat([encoded_df,
                        pd.DataFrame({'character_1': encoded_list, 'character_2': encoded_list})], # crate a couple of a character with itself
                       ignore_index=True)
# transform the data frame in an  adjacency matrix
encoded_df = pd.crosstab(encoded_df.character_1, encoded_df.character_2)
idx = encoded_df.columns.union(encoded_df.index)
encoded_df = encoded_df.reindex(index = idx, columns=idx, fill_value=0)
 # remove the identity matrix for the self loops
adjacency_matrix = encoded_df.to_numpy() - np.identity(len(encoded_df))

### 1.8
Compute the PageRank vector for the obtained graph using a damping factor of 0.85.

In [ ]:
pagerank = PageRank(damping_factor=0.85,
                    solver="piteration",
                    n_iter=1000,
                    tol=1e-06)

pagerank_vector = pagerank.fit_transform(adjacency_matrix)

Print the name and the PageRank score of the top-15 characters according to the PageRank score of their associated nodes.

[comment]: <> (#SHOW_CELL#)

In [ ]:
display(pd.DataFrame({'names': sorted_characters, 'rank': pagerank_vector}).sort_values(by="rank",
                                                                                        ascending=False).head(15))

,names,rank
162,Sir Uwaine,0.128510
112,Sir Blamore,0.059364
22,Christmas,0.053505
30,Dover,0.038614
157,Sir Tor,0.032517
28,Damosel of the Lake,0.027641
115,Sir Bors,0.025795
96,Red Knight,0.025759
31,Excalibur,0.021240
26,Dame Brisen,0.018676


<div style="page-break-after: always; visibility: hidden">
\pagebreak
</div>

### 1.9
Compute the Topic-specific PageRank vector for the obtained graph using a damping factor of 0.75, by considering as topic the *Queens*: a character belongs to the topic if its name starts with the string `Queen`.

In [ ]:
topic = 'Queen'

weights = {}
queens = []
for i, name in enumerate(sorted_characters):
  if name.startswith(topic): # select all the "long" names starting with "Queen"
    queens.append(i)
# define the weights for teleporting probability in topic-sens. pagerank
for id in queens:
  weights[id] = 1. / len(queens)



pagerank =  PageRank(damping_factor=0.75, solver='piteration', n_iter=1000, tol=1e-06)


queen_pagerank_vector = pagerank.fit_transform(adjacency_matrix, weights=weights)

Print the name and the PageRank score of the top-16 characters according to the Topic-specific PageRank score of their associated nodes.

[comment]: <> (#SHOW_CELL#)

In [ ]:
display(pd.DataFrame({'names': sorted_characters,
                      'Pagerank score': queen_pagerank_vector}).sort_values(by="Pagerank score",
                                                                            ascending=False).head(16))

,names,Pagerank score
162,Sir Uwaine,0.098612
92,Queen Isoud,0.062696
112,Sir Blamore,0.059395
93,Queen Morgan le Fay,0.059214
94,Queen of Orkney,0.057026
91,Queen Igraine,0.056662
90,Queen Guenever,0.056226
22,Christmas,0.030667
30,Dover,0.030338
157,Sir Tor,0.023053


<div style="page-break-after: always; visibility: hidden">
\pagebreak
</div>

### 1.10
Compute the Personalized PageRank vector for the obtained graph using a damping factor of 0.2 for each of the *Knights*: a character belongs to the topic if its name starts with the string `Sir`.

In [ ]:
# We take all the knights
knights = [name for name in sorted_characters if name.startswith("Sir")]

# Define the desired pagerank
pagerank =  PageRank(damping_factor=0.2, solver='piteration', n_iter=1000, tol=1e-06)


knights_pagerank_vector = {}
for knight in knights:

  # Set the personalized weights equal to 1. for each knight
  weights = {np.where(np.array(sorted_characters) == knight)[0].flat[0]: 1.}

  # Get the pagerank vector
  knights_pagerank_vector[knight] = pagerank.fit_transform(adjacency_matrix,
                                                           weights=weights)

For each of the *Knights*, print the name and the PageRank score of the top-2 characters according to the Personalized PageRank score of their associated nodes.

[comment]: <> (#SHOW_CELL#)

In [ ]:
for knight in knights:
  print(f"{knight}:")
  display(pd.DataFrame({"Character": sorted_characters,
                        "Pagerank score": knights_pagerank_vector[knight]}).sort_values(by="Pagerank score",
                                                                                        ascending=False).head(2))
  for _ in range(4): print()

Sir Accolon of Gaul:


,Character,Pagerank score
101,Sir Accolon of Gaul,0.802570
28,Damosel of the Lake,0.014832






Sir Aglovale:


,Character,Pagerank score
102,Sir Aglovale,0.803160
162,Sir Uwaine,0.017597






Sir Agravaine:


...

,Character,Pagerank score
160,Sir Uriens,1.0
0,Abbot,0.0






Sir Urre:


,Character,Pagerank score
161,Sir Urre,0.802260
162,Sir Uwaine,0.008167






Sir Uwaine:


,Character,Pagerank score
162,Sir Uwaine,1.0
0,Abbot,0.0


<div style="page-break-after: always; visibility: hidden">
\pagebreak
</div>

### 1.11
Compute Topic-specific PageRank for the graph using a damping factor of 0.2. We imagine being in an **online** context.

The Topic is *Knights* (list of characters defined in step 1.1.7)

In [ ]:
# Given that the damping factor is 0.2, which is the same of step 1.10 where we
# have calculated the personalized pagerank for the Knights topic, because of
# linearity, we can simply add all the precomputed PPR and then divide by the
# number of characters (pages) of the topic.

# Initializing the online_pagerank_vector to a np.array of all zeros
online_pagerank_vector = np.zeros(len(sorted_characters))

# We add all the precomputed personalized pagerank
for knight in knights_pagerank_vector:
  online_pagerank_vector += knights_pagerank_vector[knight]

# We divide by the number of the characters
online_pagerank_vector /= len(sorted_characters)

Print the name and the PageRank score of the top-8 characters according to the Topic-specific PageRank score of their associated nodes.

[comment]: <> (#SHOW_CELL#)

In [ ]:
display(pd.DataFrame({"Character": sorted_characters,
                      "Pagerank score": online_pagerank_vector}).sort_values(by="Pagerank score",
                                                                             ascending=False).head(8))

,Character,Pagerank score
162,Sir Uwaine,0.010716
112,Sir Blamore,0.007814
115,Sir Bors,0.006630
157,Sir Tor,0.006506
107,Sir Archade,0.006163
143,Sir Meliagrance,0.006103
116,Sir Breunor,0.006043
114,Sir Bliant,0.005972


<div style="page-break-after: always; visibility: hidden">
\pagebreak
</div>

### 1.12
Given a graph with $n$ nodes:
* Node $A$ is connected to all the other nodes.
* There are no other edges.

What will be the PageRank of node $A$?

Does the result depend on the damping factor or number of nodes $n$? If yes, describe the value of PageRank as both vary.

The Pagerank score of node A is the largest one, since it is at the center of a star graph and thus referenced by all the other nodes.\
Furthermore, from some simulations it seems that the Pagerank value of A decreases (despite always being the largest) with a convex trend w.r.t. the number of nodes and with different speed given by the damping factor value: the number of nodes to which A has a link increases as well as the number of nodes pointing to it.\
The damping factor also determines a variation of Pagerank according to a concave curve with maximum $\in (0,1)$ depending on the number of nodes: when d.f. is 1 the adjacency matrix alone will determine the Pagerank value, on the contrary, for d.f. equal to 0 there will be a uniform probability of being at certain node, thus A will take the min value.

<div style="page-break-after: always; visibility: hidden">
\pagebreak
</div>